# 縦方向のラインプロファイルによる垂直方向の平坦化

- x方向の位置は画像ごとに適当に指定
- 縦一列について、左右5画素の平均
- 係数をかけて0-255に正規化

In [ ]:
from pathlib import Path

import cv2
from japanize_matplotlib import japanize
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline
sns.set()
sns.set_style('whitegrid')
japanize()

In [ ]:
path_here = Path().cwd()
print(path_here)

In [ ]:
org_fig_dir = path_here/'07_output'
for file in org_fig_dir.glob('tire_*.png'):
    print(file)

In [ ]:
embossed_fig_dir = path_here/'12_output'
for file in embossed_fig_dir.glob('tire_*.png'):
    print(file)

In [ ]:
output_root = path_here/'13_output'
output_root.mkdir(parents=True, exist_ok=True)
print(output_root)

- xで指定した横方向の位置について、タイヤの断面を測り、左右2pxの計5pxの平均を取得
- 全体の平均(mean)とvert_profileから行単位で平坦化

In [ ]:
def correction_line_level(img, x):
    vert_profile = [None for i in range(img.shape[0])]
    for row in range(img.shape[0]):
        v = 0
        for i in range(-2, 3, 1):
            v += img[row][x+i][0]
        v /= 5
        vert_profile[row] = v
    min_profile = min(vert_profile)
    max_profile = max(vert_profile)
    mean = (min_profile + max_profile) / 2
    for row in range(img.shape[0]):
        coef = mean / vert_profile[row] if vert_profile[row] > 0 else 0
        for col in range(img.shape[1]):
            px_tmp = img[row, col] * coef
            for i in range(3):
                px_tmp[i] = min(255, px_tmp[i])
            img[row, col] = px_tmp
    return img
    

## 平坦化して保存(最後に正規化が必要だが未実施)

- 元画像を平坦化
- エンボス画像と各px比較して暗い方を採用

In [ ]:
for org_file in org_fig_dir.glob('tire_*.png'):
    print(org_file)
    org_img = cv2.imread(str(org_file))
    # print(org_img.shape)
    
    # fig = plt.figure(figsize=(60, 40))
    # plt.imshow(org_img)
    # plt.show()
    # plt.close()
    
    x = org_img.shape[1] // 2
    new_img = correction_line_level(org_img, x)
    
    # fig = plt.figure(figsize=(60, 40))
    # plt.imshow(new_img)
    # plt.show()
    # plt.close()
    
    for embossed_file in embossed_fig_dir.glob('tire_*.png'):
        if org_file.stem in str(embossed_file):
            break
    print(embossed_file)
    embossed_img = cv2.imread(str(embossed_file))
    # print(embossed_img.shape)

    # fig = plt.figure(figsize=(60, 40))
    # plt.imshow(embossed_img)
    # plt.show()
    # plt.close()

    normalizes = [org_img, embossed_img]
    arr_normalized = np.asarray(normalizes)
    normalized_max = arr_normalized.min(axis=0)

    # fig = plt.figure(figsize=(60, 40))
    # plt.imshow(normalized_max)
    # plt.show()
    # plt.close()

    cv2.imwrite(str(output_root/f'{org_file.stem}.png'), normalized_max)
    del normalized_max, normalizes, org_img, embossed_img, new_img
    